<a href="https://colab.research.google.com/github/scigeek72/portfolio/blob/main/FineTune_google_T5small_NL_to_SQL_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "torch" tensorboard

!pip install datesets -- upgrade
!pip install accelerate -- upgrade
!pip install evaluate -- upgrade
!pip install bitsandbytes  --upgrade

!pip install transformers --upgrade
!pip install trl --upgrade
!pip install peft --upgrade
!pip install rouge_score --upgrade

#!pip install git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e --upgrade
#!pip install git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f --upgrade

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install evaluate

  Using cached evaluate-0.4.2-py3-none-any.whl (84 kB)


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print(f"Is CUDA available: {torch.cuda.is_available()}\n\nNumber of CUDA device available: {torch.cuda.device_count()}")

Is CUDA available: True

Number of CUDA device available: 1


In [ ]:
#load model
model_id = "google-t5/t5-small" #t5-small has 60 million parameters...should be good for L4/T4
#model_id = "google-t5/t5-large" #t5-large has 770 million parameters
model_name = model_id.split("/")[-1]

tokenizer = AutoTokenizer.from_pretrained(model_id)
orig_model = AutoModelForSeq2SeqLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#load dataset
dataset_scc_train = load_dataset("b-mc2/sql-create-context", split="train[:80%]")
dataset_scc_test = load_dataset("b-mc2/sql-create-context", split="train[-20%:-10%]")
dataset_scc_val = load_dataset("b-mc2/sql-create-context", split="train[-10%:]")

dataset = DatasetDict()
dataset["train"] = dataset_scc_train
dataset["test"] = dataset_scc_test
dataset["val"] = dataset_scc_val

dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62862
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7857
    })
    val: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})

In [ ]:
dataset["val"][100]

{'answer': 'SELECT country FROM table_name_5 WHERE play = "medea"',
 'question': 'What country had the play Medea?',
 'context': 'CREATE TABLE table_name_5 (country VARCHAR, play VARCHAR)'}

In [ ]:
def tokenize_function(example):
  """
  Tokenize the dataset.
  Args:
    example: One row of the dataset (dict); keys: "context", "question", "answer"

  Returns:
    A dict with keys: "input_ids", "labels" as required by the t5-small model.
    check the documentation of t5-small on HF for more details: https://huggingface.co/google-t5/t5-small
  """

  start_prompt = "Tables:\n"
  middle_prompt = "\n\nQuestion:\n"
  end_prompt = "\n\nAnswer:\n"

  data_zip = zip(example["context"],example["question"])
  prompt =[start_prompt + context  + middle_prompt + question + end_prompt for context, question in data_zip]

  # for t5-small, padding="max_lenght" works, for t5-large, padding='longest' works
  example["input_ids"] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
  example["labels"] = tokenizer(example["answer"], padding="max_length", truncation=True, return_tensors="pt").input_ids
  return example

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["context", "question", "answer"])
tokenized_dataset.save_to_disk("tokenized_dataset")

Map:   0%|          | 0/62862 [00:00<?, ? examples/s]

Map:   0%|          | 0/7857 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/62862 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7857 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7858 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset.keys())
print(tokenized_dataset["train"][0].keys())
print(tokenized_dataset["train"][0]["input_ids"][:5])
print(tokenized_dataset["train"][0]["labels"][:5])

print(tokenized_dataset)

dict_keys(['train', 'test', 'val'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 205, 4386]
[3, 23143, 14196, 2847, 17161]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 62862
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 7857
    })
    val: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 7858
    })
})


# Test the Model with Zero Shot Inferencing
### Original Model, not finetuned

In [ ]:
def generate_sql(dataset, index, model, tokenizer, verbose=0):
  index = index

  question = dataset["test"][index]["question"]
  context = dataset["test"][index]["context"]
  answer = dataset["test"][index]["answer"]

  inputs = tokenizer(question, context, return_tensors="pt")

  prompt = f"""Tables:
  {context}

  Question:
  {question}

  Answer:"""

  inputs = tokenizer(prompt, return_tensors="pt")
  inputs = inputs.to(model.device)

  tokenized_output = model.generate(inputs["input_ids"], max_new_tokens=200)[0]
  output = tokenizer.decode(tokenized_output, skip_special_tokens=True)

  if verbose > 0:
    dash_line='-'.join('' for x in range(100))
    print(dash_line)
    print(f"INPUT_PROMPT:\n{prompt}")
    print(dash_line)
    print(f"BASELINE HUMAN ANSWER:\n{answer}")
    print(dash_line)
    print(f"MODEL GENERATION:\n{output}")

  return answer, output



In [ ]:
answer, output = generate_sql(dataset, 0, orig_model, tokenizer, verbose=1)

In [ ]:
answers, outputs = [],[]
for idx in range(2):
  answer, output = generate_sql(dataset, idx, orig_model, tokenizer, verbose=1)
  answers.append(answer)
  outputs.append(output)

print(answers,"\n\n", outputs)

# Fine-Tuning

In [ ]:
#We will load the model and the tokenizer once again and fine tune it
#(so that we can compare the output from both the models)

finetune_model = AutoModelForSeq2SeqLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
output_dir = f"./{model_name}-text-to-sql"

#change the outputs of other arguments once it can recreate the output
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-3,
    num_train_epochs=3, #changed from 2
    per_device_train_batch_size=16, #changed from 8
    per_device_eval_batch_size=16,  #changed from 8
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=500,
)



In [ ]:
trainer = Trainer(
    model=finetune_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
)

trainer_stat = trainer.train()



Step,Training Loss,Validation Loss
500,0.052600,0.033867
1000,0.024000,0.016765
1500,0.020600,0.014021
2000,0.019700,0.011667
2500,0.018500,0.010353
3000,0.014100,0.009238
3500,0.014000,0.008636
4000,0.011600,0.008098
4500,0.009400,0.007955
5000,0.011700,0.007392


In [ ]:
trainer.save_model(output_dir+f"/fine_tuned_{model_name}_epoch_2")
#tokenizer.save_toke(output_dir/f"fine-tuned-{model_name}")

# Test the fine tuned model

In [ ]:

finetuned_model_answers = []
for index in range(10):
  answer, output = generate_sql(dataset, index, finetune_model, tokenizer)
  finetuned_model_answers.append(output)



In [ ]:
finetuned_model_answers

['SELECT date FROM table_name_11 WHERE away_team = "essendon"',
 'SELECT date FROM table_name_49 WHERE home_team = "north melbourne"',
 'SELECT record FROM table_name_72 WHERE date = "september 1"',
 'SELECT loser FROM table_name_48 WHERE date = "september 1"',
 'SELECT 1988 FROM table_name_88 WHERE 1987 = "1r"',
 'SEL 1981 FROM table_name_97 WHERE tournament = "the tournament of championships of tennis"',
 'SELECT name FROM table_name_86 WHERE opened = 2004 AND park = "bobbejaanland"',
 'SELECT status FROM table_name_89 WHERE opened = 2008 AND model = "junior coaster"',
 'SELECT model FROM table_name_16 WHERE status = "operating" AND name = "thor\'s hammer"',
 'SELECT name FROM table_name_80 WHERE model = "euro-fighter" AND opened = "2011"']

In [ ]:
#index = 7850#0

original_model_answers = []
human_baseline_answers = []

for index in range(10):
  answer, output = generate_sql(dataset, index, orig_model, tokenizer)
  original_model_answers.append(output)
  human_baseline_answers.append(answer)


In [ ]:
original_model_answers

['Question',
 'Question',
 'Question',
 'True',
 'Question',
 'Question: What is the 1981 value at the Tournament of Wimbledon? Answer: What is the 1981 value at the Tournament of Wimbledon?',
 'True',
 'True',
 'True',
 'True']

In [ ]:
human_baseline_answers

['SELECT date FROM table_name_11 WHERE away_team = "essendon"',
 'SELECT date FROM table_name_49 WHERE home_team = "north melbourne"',
 'SELECT record FROM table_name_72 WHERE date = "september 1"',
 'SELECT loss FROM table_name_48 WHERE date = "september 1"',
 'SELECT 1988 FROM table_name_88 WHERE 1987 = "1r"',
 'SELECT 1981 FROM table_name_97 WHERE tournament = "wimbledon"',
 'SELECT name FROM table_name_86 WHERE opened = "2004" AND park = "bobbejaanland"',
 'SELECT status FROM table_name_89 WHERE opened = "2008" AND model = "junior coaster"',
 'SELECT model FROM table_name_16 WHERE status = "operating" AND name = "thor\'s hammer"',
 'SELECT name FROM table_name_80 WHERE model = "euro-fighter" AND opened = "2011"']

# Compute ROGUE scores (measure)

In [ ]:
# Only do 25 datapoints

# questions = dataset['test']['question'][:1]
# contexts = dataset['test']['context'][:1]
# answers = dataset['test']['answer'][:1]

# original_model_answers = []
# finetuned_model_answers = []
# human_baseline_answers = []

# answers = []

# for idx in range(2):
#   question = dataset["test"][idx]["question"]
#   context = dataset["test"][idx]["context"]
#   answer = dataset["test"][idx]["answer"]
#   #first generate prompt
#   prompt = f"""Tables:
#   {context}

#   Question:
#   {question}

#   Answer:"""
#   orig_input_ids = tokenizer(prompt, return_tensors="pt")
#   orig_input_ids = orig_input_ids.to(orig_model.device)

#   # print(prompt)
#   # print(idx,"  ", orig_input_ids['input_ids'])

#   original_tokenized_output = orig_model.generate(orig_input_ids["input_ids"],max_new_tokens=200)[0]
#   original_text_output = tokenizer.decode(original_tokenized_output[0], skip_special_tokens=True)
#   print(original_text_output)

#   original_model_answers.append(original_text_output)

#   finetune_input_ids = tokenizer(prompt, return_tensors="pt")
#   finetune_input_ids = finetune_input_ids.to(finetune_model.device)



#   finetuned_tokenized_output = finetune_model.generate(finetune_input_ids["input_ids"], max_new_tokens=200)[0]
#   finetuned_text_output = tokenizer.decode(finetuned_tokenized_output[0], skip_special_tokens=True)
#   finetuned_model_answers.append(finetuned_text_output)

#   human_baseline_answers.append(answer)

zipped_answers = list(zip(human_baseline_answers, original_model_answers, finetuned_model_answers))
answerDF = pd.DataFrame(zipped_answers, columns=['human_baseline_answer', 'original_model_answers', 'finetuned_model_answers'])
answerDF


,human_baseline_answer,original_model_answers,finetuned_model_answers
0,SELECT date FROM table_name_11 WHERE away_team...,Question,SELECT date FROM table_name_11 WHERE away_team...
1,SELECT date FROM table_name_49 WHERE home_team...,Question,SELECT date FROM table_name_49 WHERE home_team...
2,SELECT record FROM table_name_72 WHERE date = ...,Question,SELECT record FROM table_name_72 WHERE date = ...
3,"SELECT loss FROM table_name_48 WHERE date = ""s...",True,"SELECT loser FROM table_name_48 WHERE date = ""..."
4,"SELECT 1988 FROM table_name_88 WHERE 1987 = ""1r""",Question,"SELECT 1988 FROM table_name_88 WHERE 1987 = ""1r"""
5,SELECT 1981 FROM table_name_97 WHERE tournamen...,Question: What is the 1981 value at the Tourna...,SEL 1981 FROM table_name_97 WHERE tournament =...
6,SELECT name FROM table_name_86 WHERE opened = ...,True,SELECT name FROM table_name_86 WHERE opened = ...
7,SELECT status FROM table_name_89 WHERE opened ...,True,SELECT status FROM table_name_89 WHERE opened ...
8,SELECT model FROM table_name_16 WHERE status =...,True,SELECT model FROM table_name_16 WHERE status =...
9,"SELECT name FROM table_name_80 WHERE model = ""...",True,"SELECT name FROM table_name_80 WHERE model = ""..."


In [ ]:
rogue = evaluate.load("rouge")

rogue.compute(predictions= finetuned_model_answers,
                                        references= human_baseline_answers,
                                        use_aggregator=True,
                                        use_stemmer=True)

{'rouge1': 0.9508695652173913,
 'rouge2': 0.9349206349206349,
 'rougeL': 0.9508695652173913,
 'rougeLsum': 0.9508695652173913}

In [ ]:
# calculate rogue score
rogue = evaluate.load("rouge")
finetuned_model_results = rogue.compute(predictions= finetuned_model_answers,
                                        references= human_baseline_answers,
                                        #use_aggregators=True,
                                        #use_stemmers=True
                                        )
original_model_results = rogue.compute(predictions= original_model_answers,
                                       references = human_baseline_answers,
                                       #use_aggregators=True,
                                       #use_stemmers=True
                                       )

print(f"Original Model:\n\t   {original_model_results}\n\n")
print(f"Finetuned Model:\n\t  {finetuned_model_results}")



Original Model:
	   {'rouge1': 0.01935483870967742, 'rouge2': 0.0, 'rougeL': 0.01935483870967742, 'rougeLsum': 0.01935483870967742}


Finetuned Model:
	  {'rouge1': 0.9508695652173913, 'rouge2': 0.9349206349206349, 'rougeL': 0.9508695652173913, 'rougeLsum': 0.9508695652173913}


### Epoch=2, batch=8 results

Original Model:
	   {'rouge1': 0.01935483870967742, 'rouge2': 0.0, 'rougeL': 0.01935483870967742, 'rougeLsum': 0.01935483870967742}


Finetuned Model:
	  {'rouge1': 0.9637023593466425, 'rouge2': 0.9262443438914026, 'rougeL': 0.9517241379310345, 'rougeLsum': 0.9491703396422091}

### Epoch=3, batch=8 results

Original Model:
	   {'rouge1': 0.01935483870967742, 'rouge2': 0.0, 'rougeL': 0.01935483870967742, 'rougeLsum': 0.01935483870967742}


Finetuned Model:
	  {'rouge1': 0.9766666666666668, 'rouge2': 0.9492063492063492, 'rougeL': 0.9700000000000001, 'rougeLsum': 0.97}


### Epoch=3, batch=16 results
Original Model:
	   {'rouge1': 0.01935483870967742, 'rouge2': 0.0, 'rougeL': 0.01935483870967742, 'rougeLsum': 0.01935483870967742}


Finetuned Model:
	  {'rouge1': 0.9508695652173913, 'rouge2': 0.9349206349206349, 'rougeL': 0.9508695652173913, 'rougeLsum': 0.9508695652173913}
